# IMDb

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/6/69/IMDB_Logo_2016.svg/640px-IMDB_Logo_2016.svg.png?1623833316814">

## Descriptions

    - IMDb (an acronym for Internet Movie Database) is an online database of information related to films, television programs, home videos, video games, and streaming content online – including cast, production crew and personal biographies, plot summaries, trivia, ratings, and fan and critical reviews. An additional fan feature, message boards, was abandoned in February 2017. Originally a fan-operated website, the database is now owned and operated by IMDb.com, Inc., a subsidiary of Amazon.

    - As of December 2020, IMDb has approximately 7.5 million titles (including episodes) and 10.4 million personalities in its database, as well as 83 million registered users.

    - IMDb began as a movie database on the Usenet group "rec.arts.movies" in 1990 and moved to the web in 1993.

    - Information ref: https://www.wikiwand.com/en/IMDb

    - Now, we're interested in what's going on and finding insight by given data from "https://relational.fit.cvut.cz/dataset/IMDb"

## Objective
- Building movies recommendation based on personal information from user. 

**NOTED that we haven't currently planned to build model yet
`
In this project, we only prepare data for modeling**

## Proceduces for this project

    1 Business Understanding
    2 Data Preparation
        - Collecting
        - Cleaning
    3 Data Understanding
    4 Describing Data
        - EDA

## Additional Libraries
- pymysql
- pandas_profiling

In [0]:
# ! pip install pymysql

In [0]:
# ! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip 

## Entity Relational Diagram

    A Diagram display the relations between databases is shown here

<img src="https://relational.fit.cvut.cz/assets/img/datasets-generated/imdb_ijs.svg">

<img src="https://media1.tenor.com/images/871791f2c3b6a691428d5eb69001647e/tenor.gif?itemid=16088453" width="350" height="350">

## Data Preparation section

### Collecting data

- Retrieve tables from given database

In [1]:
# Inspects all all tables, are still available, from database

import pymysql.cursors

hostname =  "relational.fit.cvut.cz"
port = 3306
username = "guest"
password = "relational"
db = "imdb_ijs"

connections = pymysql.connect(
    host=hostname,
    port=port,
    user=username,
    password=password,
    db=db,
    charset="utf8mb4",
    cursorclass=pymysql.cursors.DictCursor
)

cursor = connections.cursor()
cursor.execute("show tables;")
tables = cursor.fetchall()
cursor.close()

for ele in tables:
    print(ele)

{'Tables_in_imdb_ijs': 'actors'}
{'Tables_in_imdb_ijs': 'directors'}
{'Tables_in_imdb_ijs': 'directors_genres'}
{'Tables_in_imdb_ijs': 'movies'}
{'Tables_in_imdb_ijs': 'movies_directors'}
{'Tables_in_imdb_ijs': 'movies_genres'}
{'Tables_in_imdb_ijs': 'roles'}


In [2]:
# Import(1)
# Extracts tables from database to pandas dataframe

import pandas as pd

def extract_dataframe(table_name: str):
    with connections.cursor() as cursor:
        # select all from given table name without any conditions
        select = "SELECT * FROM " + table_name
        cursor.execute(select)
        result = cursor.fetchall()
        return pd.DataFrame(result)

raw_actors = extract_dataframe("actors")
raw_directors = extract_dataframe("directors")
raw_directorsGen = extract_dataframe("directors_genres")
raw_movies = extract_dataframe("movies")
raw_moviesDirec = extract_dataframe("movies_directors")
raw_moviesGen = extract_dataframe("movies_genres")
raw_roles = extract_dataframe("roles")

# Assigning name
dfs_name = {
    "Actors" : raw_actors,
    "Directors" : raw_directors,
    "Directors-genres" : raw_directorsGen,
    "Movies" : raw_movies,
    "Movies-directors" : raw_moviesDirec,
    "Movies-genres" : raw_moviesGen,
    "Roles" : raw_roles 
}

list_raw = [
    raw_actors,
    raw_directors,
    raw_directorsGen,
    raw_movies,
    raw_moviesDirec,
    raw_moviesGen,
    raw_roles
]

In [0]:
# Exporting raw tables

# raw_actors.to_csv("actors.csv")
# raw_directors.to_csv("directors.csv")
# raw_directorsGen.to_csv("directors_genres.csv")
# raw_movies.to_csv("movies.csv")
# raw_moviesDirec.to_csv("movies_directors.csv")
# raw_moviesGen.to_csv("movies_genres.csv")
# raw_roles.to_csv("roles.csv")

In [12]:
# Print the first few row of each

for name_df, value in dfs_name.items():
    print(name_df)
    print(value.head())
    print()

Actors
   id first_name           last_name gender
0   2    Michael  'babeepower' Viera      M
1   3       Eloy         'Chincheta'      M
2   4   Dieguito         'El Cigala'      M
3   5    Antonio    'El de Chipiona'      M
4   6       José        'El Francés'      M

Directors
   id first_name    last_name
0   1       Todd            1
1   2        Les  12 Poissons
2   3    Lejaren     a'Hiller
3   4       Nian            A
4   5   Khairiya    A-Mansour

Directors-genres
   director_id        genre  prob
0            2        Short   1.0
1            3        Drama   1.0
2            5  Documentary   1.0
3            6        Drama   1.0
4            6        Short   1.0

Movies
   id               name  year  rank
0   2                  $  1971   6.4
1   6    $1,000,000 Duck  1971   5.0
2  11  $1000 a Touchdown  1939   6.7
3  15                $30  1999   7.5
4  18            $40,000  1996   9.6

Movies-directors
   director_id  movie_id
0            1    378879
1            2    

### Data Cleaning
    1 Validate each columns are in the right type
    2 Remove missing values
    3 Adjust values error (the value that makes no sense)
    4 Remove duplicates rows
    5 Standardize data (if possible)

<img src="https://media1.tenor.com/images/5237c33065a69a734309974a8b7f029b/tenor.gif?itemid=15522288" width="200" height="200">

#### Correcting data types & Finding missing values
- Inspect all data type in each columns that has been stored correctly based on **Data Dictionary**
- Indentify the number of missing values

In [5]:
# Looking basic information of each tables
# The basic information can tell us the type of each columns, null object

for name_df, df in dfs_name.items():
    print(name_df)
    print(df.info())
    print()

Actors
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 817718 entries, 0 to 817717
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id          817718 non-null  int64 
 1   first_name  817718 non-null  object
 2   last_name   817718 non-null  object
 3   gender      817718 non-null  object
dtypes: int64(1), object(3)
memory usage: 25.0+ MB
None

Directors
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86880 entries, 0 to 86879
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          86880 non-null  int64 
 1   first_name  86880 non-null  object
 2   last_name   86880 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.0+ MB
None

Directors-genres
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156562 entries, 0 to 156561
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -

##### Report

**Correcting data types**

- We inspected that **all columns are in the right type**
- e.g. all id columns are in integer type, rank is in float type, names or genres are in object typr

**Missing values**

- There are massively null valus in Movies dataframe (321024)

#### Remove missing values

- Remove on [ movies table ]

In [13]:
# Remove row that contains missing value

droppedNull_movies = raw_movies.dropna()

droppedNull_movies =  droppedNull_movies.reset_index(drop=True)

# Update dictionary
dfs_name["Movies"] = droppedNull_movies

droppedNull_movies.head(10)

,id,name,year,rank
0,2,$,1971,6.4
1,6,"$1,000,000 Duck",1971,5.0
2,11,$1000 a Touchdown,1939,6.7
3,15,$30,1999,7.5
4,18,"$40,000",1996,9.6
5,24,"$50,000 Climax Show, The",1975,2.6
6,31,$pent,2000,4.3
7,33,$windle,2002,5.4
8,36,'15',2002,6.8
9,38,'38,1987,6.7


In [14]:
# check all id column of each table are unique (no duplicate id)

ids_col = ["id", "director_id", "movie_id", "actor_id"]
for name, df in dfs_name.items():
    # find id column
    for id_col in df.columns:
        if id_col in ids_col:
            print(name, "on column", id_col)
            print("ID unique:",df[id_col].duplicated().any())
            print()
            break

Actors on column id
ID unique: False

Directors on column id
ID unique: False

Directors-genres on column director_id
ID unique: True

Movies on column id
ID unique: False

Movies-directors on column director_id
ID unique: True

Movies-genres on column movie_id
ID unique: True

Roles on column actor_id
ID unique: True



#### Adjust values error
- Check and change (if neccessary) the values whether are in the right format

In [15]:
# basic statistics
# Continue to check all dataframe

droppedNull_movies.describe()

,id,year,rank
count,67245.000000,67245.000000,67245.000000
mean,189297.063945,1976.594275,5.874239
std,108991.555948,23.854135,1.622712
min,2.000000,1890.000000,1.000000
25%,94218.000000,1960.000000,4.900000
50%,188803.000000,1984.000000,6.000000
75%,284587.000000,1997.000000,7.000000
max,378614.000000,2004.000000,9.900000


##### Result

**Movies dataframe**

- We ignore id column (already checked id column)
- Year initially collected since 1890 to 2004 -> (make sense)
- Rank (rating) min is 1 through 9.9 (max) -> (make sense)